### I wanted to build some Tableau visualizations with this publically available dataset from Kaggle: 
https://www.kaggle.com/datasets/vnxiclaire/bobabayarea/data

### The above is a 2018 sample of bubble tea, or boba, shops in the Bay Area, collected via the Yelp API.
### Addtionally, there is a 2024 dataset contributed by a different user, but following the same basic methodology:
https://www.kaggle.com/datasets/willatran/boba-shops-in-the-bay-area-2024/data

In [841]:
import pandas as pd
import os

boba = pd.read_csv(r'E:\AnalystBuilderTestFolder\BayBoba\bayarea_boba_spots_2018.csv')
pd.set_option('display.max_rows', None)
#I checked Nulls and Datatypes with info(), but commenting it out to reduce bloat. No action needed.
#boba.info()

In [843]:
#To illustrate the column structure in the raw input.
boba.head(2)

,Unnamed: 0,id,name,rating,address,city,lat,long
0,0,99-tea-house-fremont-2,99% Tea House,4.5,3623 Thornton Ave,Fremont,37.562950,-122.010040
1,1,one-tea-fremont-2,One Tea,4.5,46809 Warm Springs Blvd,Fremont,37.489067,-121.929414


In [845]:
#The first column of the dataset, 'Unnamed: 0', is just an index. (and redundant)
#the second column is Yelp's ID for the location, which is also unnecessary.
boba = boba.drop(boba.columns[[0, 1]], axis=1)

### The data cleaning is needed in the 'name' column. I'm interested in the stores differentiated by brand. Unfortunately, the name data is dirty for a variety of reasons:

### Reason 1: Appending a location. In this dataset, this most commonly occurs with: '[Name] - [Location]'
### Such as:

In [849]:
boba = boba.sort_values(by=['name'])
boba[boba['name'].str.contains('- ')].head(2)

,name,rating,address,city,lat,long
369,Coupa Cafe - Green Library,3.5,571 Escondido Mall,Stanford,37.426249,-122.167068
360,Coupa Café - Lytton,3.5,111 Lytton Ave,Palo Alto,37.444472,-122.165222


### However, there's exceptions to this as well, where this specific shop uses a hyphen to append the English name to the Vietnamese name.

In [852]:
boba[boba['name'].str.contains('- ')].iloc[2:4]

,name,rating,address,city,lat,long
498,Pho Huynh Hiep 5 - Kevin's Noodle House,3.5,1833 Willow Pass Rd,Concord,37.975140,-122.039010
500,Pho Huynh Hiep 6 - Kevin's Noodle House,4.0,2034 N Main St,Walnut Creek,37.907986,-122.064301


In [854]:
boba.loc[boba['name'].str.contains('Huynh Hiep'), 'name'] = 'Pho Huynh Hiep'
boba['name'] = boba['name'].str.split(pat=' - ', n=1, expand=True)[0]

### Reason 2: incorrect whitespace inputs. 

In [857]:
boba['name'].iloc[234:236]

220    Menchie's  Frozen Yogurt
550     Menchie's Frozen Yogurt
Name: name, dtype: object

In [859]:
boba['name'] = boba['name'].str.replace('  ', ' ')

### Going forward I will copy in an example in this markdown section, instead of illustrating with the dataframe, for brevity. Since the remaining reasons don't share an algorithimic cleaning process.
### Reason 3: Brand Casing Variance
ICICLES

Icicles
### Reason 4: Brand Punctuation
Mr Green Bubble

Mr. Green Bubble
### Remaing Reasons: An assorted group of the same brand, but the name has addition information appended. This could be location without hyphenation, a joint restaurant venture, or a new location with a numerical increment.
T4

T4 Cupertino

Vampire Penguin

Vampire Penguin featuring Jastea

World Wrapps

World Wrapps 2
### Unfortunately, this data just needs to be manually addressed. Fortunately the dataset is of a manageable size.

In [862]:
#Normal Replacements
regexList = ['Happy Lemon', 'Quickly', 'STEEP', 'Sharetea', 'T4', 'Ten Ren Tea', 'Vampire Penguin', 'World Wrapps', 'Blackball Desserts']
replaceList = regexList[:]

#Special Cases
regexList += ['Icicles', 'Green Bubble']
replaceList += ['ICICLES', 'Mr. Green Bubble']

for regex, replacement in zip(regexList, replaceList):
    boba.loc[boba['name'].str.contains(regex), 'name'] = replacement

In [864]:
boba = boba.sort_values(by=['name'])

#Adding year to differentiate after I concatenate in Tableau
boba['year'] = 2018

In [871]:
if os.path.exists(r'E:\AnalystBuilderTestFolder\BayBoba\2018_Cleaned_BayBoba.csv'):
    boba.to_csv(r'E:\AnalystBuilderTestFolder\BayBoba\2018_Cleaned_BayBoba.csv', header = True)
else:
    boba.to_csv(r'E:\AnalystBuilderTestFolder\BayBoba\2018_Cleaned_BayBoba.csv')

### The 2018 data is clean, time to export and repeat the process for 2024.

In [873]:
boba2 = pd.read_csv(r'E:\AnalystBuilderTestFolder\BayBoba\updated_bayarea_boba_spots_2024.csv')
#Dropping Index and Id again.
boba2 = boba2.drop(boba2.columns[[0, 1]], axis=1)
boba2 = boba2.sort_values(by=['name'])

#boba2.info()

In [875]:
boba2['name'] = boba2['name'].str.replace('  ', ' ')
boba2['name'] = boba2['name'].str.split(pat=' - ', n=1, expand=True)[0]

In [877]:
#Addressing the one outlier, where the ' - ' was not a '[Name] - [Location]' combo, but rather the actual brand name.
#Note that B&B Boba and Banh Mi is a separate entity from b&b Bahn Mi and Boba.
boba2.loc[boba2['name'].str.contains('b&b'), 'name'] = 'b&b - Banh Mi & Boba'

In [879]:
#Normal Replacements
regexList = ['Happy Lemon', 'Quickly', 'STEEP', 'Sharetea', 'T4', 'Tea&Poke', 'TenZenTea', 'Sunright Tea Studio', 'Teaspoon', 'Rabbit Rabbit']
regexList += ['Mochinut', 'MandRo Teahouse', 'Mr Sun', 'Lilikoi', 'Feng Cha', 'CoCo', 'BAMBU', 'Boba Guys', 'Bober Tea']
              
replaceList = regexList[:]

#Special Cases
regexList += ['Icicles', 'Green Bubble', 'Rare', 'Quali[T|t]ea', 'ZERO *&', 'Taiwan Fruit', 'Ten *(Ren|ren)', 'Gong', 'Devil', 'Big.Bang']
replaceList += ['ICICLES', 'Mr. Green Bubble', 'RareTea', 'QualiTea', 'ZERO&', 'Yi Fang Taiwan Fruit', 'Ten Ren Tea', 'Gong Cha', 'Devil & Angel Desserts']
replaceList += ['Big Bang Boba & Chicken']

#This specific store seems to be unaffiliated with T4, the Taiwanese brand. 
boba2.loc[boba2['name'].str.contains('T4 a cup of tea'), 'name'] = 'knockoff'

for regex, replacement in zip(regexList, replaceList):
    boba2.loc[boba2['name'].str.contains(regex), 'name'] = replacement

boba2.loc[boba2['name'].str.contains('knockoff'), 'name'] = 'T4 a cup of tea'

C:\Users\wush1\AppData\Local\Temp\ipykernel_19604\1626555646.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  boba2.loc[boba2['name'].str.contains(regex), 'name'] = replacement


In [881]:
boba2 = boba2.sort_values(by=['name'])
boba2['year'] = 2024

In [883]:
if os.path.exists(r'E:\AnalystBuilderTestFolder\BayBoba\2024_Cleaned_BayBoba.csv'):
    boba2.to_csv(r'E:\AnalystBuilderTestFolder\BayBoba\2024_Cleaned_BayBoba.csv', header = True)
else:
    boba2.to_csv(r'E:\AnalystBuilderTestFolder\BayBoba\2024_Cleaned_BayBoba.csv')